# **Analyzing Drug Dispensation Trends in Calgary and Edmonton from 2010 to 2018**

![brain-3829057_960_720](brain-3829057_960_720.jpg)


### Analyzing data related to anti-depressant and anti-anxiety medication usage can provide some interesting information, for this data set we can see which drugs were more popularily prescribed over the years. 

### This particular data set includes information on specific drug types, age groups, gender, and dispensation rates. As someone with a keen interest in health-related data, I was particularly interested in exploring this dataset to uncover any noteworthy parameters related to why certain medications were used than others. 

## Overview of the data set

In [1]:
--Scope the overview 
SELECT *
FROM Dispensation.csv;


SyntaxError: invalid syntax (3202232539.py, line 1)

## What are all the names of drug types and how many are there?

In [78]:
-- Seeing the types of drugs and the total count by the # of rows listed
SELECT drug_type, COUNT(DISTINCT drug_type) AS TOTAL --Count all 
FROM Dispensation.csv
GROUP BY drug_type;


,drug_type,TOTAL
0,ALPRAZOLAM,1
1,AMITRIPTYLINE,1
2,BROMAZEPAM,1
3,BUPROPION,1
4,BUSPIRONE,1
5,CHLORDIAZEPOXIDE,1
6,CITALOPRAM,1
7,CLOBAZAM,1
8,CLOMIPRAMINE,1
9,CLORAZEPATE POTASSIUM,1


## We want to find which city has more people getting prescribed drugs

In [43]:
SELECT city, ROUND(SUM(total_dispensation),0) as total
FROM Dispensation.csv
GROUP BY city
ORDER BY total DESC;




,city,total
0,Edmonton,15754079.0
1,Calgary,10728255.0


## We also want to find which drug is most prescribed between the 2 cities

In [12]:
SELECT drug_type, SUM(total_dispensation)as total_dispensation
FROM Dispensation.csv
WHERE city IN ('Edmonton', 'Calgary')
GROUP BY drug_type
ORDER BY total_dispensation DESC
LIMIT 5;







,drug_type,total_dispensation
0,CITALOPRAM,3288945.0
1,VENLAFAXINE,3211742.0
2,LORAZEPAM,2588397.0
3,ESCITALOPRAM,2319382.0
4,TRAZODONE,2275491.0


## Let's see how many of each of the drugs are prescribed by the years

In [68]:
SELECT d.drug_type, y.year, SUM(d.total_dispensation) as total_dispensation
FROM Dispensation.csv as d
JOIN ( --Join the table and subquery
    SELECT DISTINCT year -- Add subquery to enable the distinct year
    FROM Dispensation.csv)
	y ON d.year = y.year
GROUP BY d.drug_type, y.year
ORDER BY y.year ASC, total_dispensation DESC;




,drug_type,year,total_dispensation
0,CITALOPRAM,2010,341800.0
1,VENLAFAXINE,2010,325467.0
2,LORAZEPAM,2010,214877.0
3,TRAZODONE,2010,138972.0
4,BUPROPION,2010,134408.0
...,...,...,...
305,DESIPRAMINE,2018,1869.0
306,PHENELZINE,2018,702.0
307,VILAZODONE,2018,246.0
308,CLORAZEPATE POTASSIUM,2018,0.0


## We can narrow it down, and Find the most prescribed drug by year 

In [14]:
SELECT drug_type, year, total_dispensation
FROM (
    SELECT drug_type, year, SUM(total_dispensation) as total_dispensation, 
           ROW_NUMBER() OVER (PARTITION BY year ORDER BY SUM(total_dispensation) DESC) as rank  --Using a window function to assign rank to the row with the highest sum
    FROM Dispensation.csv
    GROUP BY drug_type, year) 
WHERE rank = 1 --Showing the top most prescribed drug
ORDER BY year; 



,drug_type,year,total_dispensation
0,CITALOPRAM,2010,341800.0
1,CITALOPRAM,2011,409436.0
2,CITALOPRAM,2012,371338.0
3,CITALOPRAM,2013,380224.0
4,CITALOPRAM,2014,345325.0
5,CITALOPRAM,2015,361362.0
6,VENLAFAXINE,2016,382665.0
7,ESCITALOPRAM,2017,409142.0
8,ESCITALOPRAM,2018,432151.0


## Now lets look at the age groups, let's find out how many despositions there were by age group

In [15]:
SELECT age, sum(total_dispensation) as total
FROM Dispensation.csv
where total_dispensation > 1
group by age
order by total;

,age,total
0,0-4,26003.0
1,5-9,65078.0
2,10-14,208655.0
3,15-19,551586.0
4,20-24,845166.0
5,90+,971669.0
6,75-79,1033736.0
7,70-74,1082372.0
8,85-89,1112926.0
9,80-84,1165337.0


## Here we can see the type of drug most prescribed by age group

In [2]:
SELECT age, drug_type, total_dispensation
FROM (
  SELECT age, drug_type, SUM(total_dispensation) AS total_dispensation, 
	--Using window function to gather values in specific row
         ROW_NUMBER() OVER (PARTITION BY age ORDER BY SUM(total_dispensation) DESC) AS rn
  FROM Dispensation.csv
  WHERE total_dispensation > 1 -- Don't include any rows with 0
  GROUP BY age, drug_type
) 
WHERE rn = 1
ORDER BY total_dispensation desc;

SyntaxError: invalid syntax (1402483680.py, line 1)

## We can find the total and average dispensations by city and drug_type, while filtering with the where clause 
## 

In [48]:
SELECT city, drug_type,SUM(total_dispensation) AS total_dispensed, ROUND(AVG(total_dispensation),0) AS avg
FROM Dispensation.csv
WHERE age = '20-24' AND year = '2018' --Add a where clause to filter the data on the specific results
GROUP BY city, drug_type;


,city,drug_type,total_dispensed,avg
0,Calgary,ALPRAZOLAM,292.0,146.0
1,Calgary,AMITRIPTYLINE,1872.0,936.0
2,Calgary,BROMAZEPAM,39.0,20.0
3,Calgary,BUPROPION,4245.0,2123.0
4,Calgary,BUSPIRONE,475.0,238.0
...,...,...,...,...
56,Edmonton,TRANYLCYPROMINE,0.0,0.0
57,Edmonton,TRAZODONE,5352.0,2676.0
58,Edmonton,TRYPTOPHAN,80.0,40.0
59,Edmonton,VENLAFAXINE,7633.0,3817.0


## Let's investigate whether males or females are prescribed medication more frequently

In [4]:
SELECT sex, SUM(total_dispensation) as total
FROM Dispensation.csv
GROUP BY sex;


SyntaxError: invalid syntax (1776333575.py, line 1)

## We can take this findings and put them into a percentage

In [54]:
SELECT sex,
ROUND((CAST(SUM(total_dispensation) AS decimal) / CAST(SUM(SUM(total_dispensation)) OVER() AS decimal)) * 100, 2) AS percentage --Using subquery to calculate total_dispensation for sex, the outer query then calculate the total sum from the inner query using the over function.
FROM Dispensation.csv
GROUP BY sex;

,sex,percentage
0,F,63.62
1,M,36.38


## Here we can separate the findings by gender, and we notice Calgary has less despositons

In [49]:
SELECT city,sex, sum(total_dispensation) as total
FROM Dispensation.csv
GROUP BY city,sex
ORDER BY total DESC;


,city,sex,total
0,Edmonton,F,9771436.0
1,Calgary,F,7076968.0
2,Edmonton,M,5982643.0
3,Calgary,M,3651287.0


## We can find out what the most common drug despositions are between the 2 cities using sex

In [50]:
SELECT city,sex, drug_type , sum(total_dispensation) as total
FROM Dispensation.csv
GROUP BY city,sex,drug_type
ORDER BY total DESC
LIMIT 10;

,city,sex,drug_type,total
0,Edmonton,F,CITALOPRAM,1255520.0
1,Edmonton,F,VENLAFAXINE,1249753.0
2,Edmonton,F,LORAZEPAM,1050397.0
3,Calgary,F,VENLAFAXINE,912968.0
4,Edmonton,F,TRAZODONE,861575.0
5,Calgary,F,CITALOPRAM,856328.0
6,Edmonton,F,ESCITALOPRAM,774149.0
7,Edmonton,M,CITALOPRAM,767587.0
8,Calgary,F,ESCITALOPRAM,719594.0
9,Edmonton,F,BUPROPION,688926.0


## Here we can take the standard scores and put them into a separate category called rating using the actual metric used by Alberta Health to compare regions.

In [51]:
SELECT total_dispensation, standard_score,
  CASE 
    WHEN total_dispensation = 0 THEN 'NA' --Using Case statements to achieve desired output
    WHEN standard_score > 2  THEN 'Significantly Higher than AVG' 
    WHEN standard_score BETWEEN 1 AND 2 THEN 'Higher than AVG' 
    WHEN standard_score BETWEEN -1 AND 1 THEN 'Average' 
    WHEN standard_score BETWEEN -2 AND -1 THEN 'Lower than AVG'
    WHEN standard_score < -2 THEN 'Significantly Lower than AVG'
    ELSE 'NA' 
  END AS rating
FROM Dispensation.csv;

,total_dispensation,standard_score,rating
0,0,0.000000,NA
1,0,-0.367835,NA
2,0,0.000000,NA
3,0,0.000000,NA
4,0,0.000000,NA
...,...,...,...
20452,1159,0.055434,Average
20453,0,0.000000,NA
20454,4060,4.173140,Significantly Higher than AVG
20455,797,-0.229411,Average


## Here we find which age groups related to drug types and years that had significantly higher than average despositions.

In [16]:
SELECT drug_type,year,age, rating, 
FROM (
  SELECT drug_type,year,age,standard_score,    --Adding case statment to subquery to find specific drug_type,year,age
    CASE 
      WHEN standard_score > 2  THEN 'Significantly Higher than AVG' 
      WHEN standard_score BETWEEN 1 AND 2 THEN 'Higher than AVG' 
      WHEN standard_score BETWEEN -1 AND 1 THEN 'Average' 
      WHEN standard_score BETWEEN -2 AND -1 THEN 'Lower than AVG'
      WHEN standard_score < -2 THEN 'Significantly Lower than AVG'
      ELSE 'NA' 
    END AS rating
  FROM Dispensation.csv
) AS subquery
WHERE rating = 'Significantly Higher than AVG';

,drug_type,year,age,rating
0,CLOBAZAM,2010,0-4,Significantly Higher than AVG
1,CLOBAZAM,2010,0-4,Significantly Higher than AVG
2,LORAZEPAM,2010,0-4,Significantly Higher than AVG
3,TRYPTOPHAN,2010,10-14,Significantly Higher than AVG
4,DESVENLAFAXINE,2010,20-24,Significantly Higher than AVG
...,...,...,...,...
2338,NORTRIPTYLINE,2018,65-69,Significantly Higher than AVG
2339,NORTRIPTYLINE,2018,70-74,Significantly Higher than AVG
2340,NORTRIPTYLINE,2018,75-79,Significantly Higher than AVG
2341,NORTRIPTYLINE,2018,80-84,Significantly Higher than AVG


# **Key insights**

From this data I have gained some valuable insights, and had several of my questions answered by working with this data. I have also made a visual interactive dashboard with **Tableau** showing this.


- I found that Females were nearly double that of their male counterparts in obtaining prescription anti anxiey / anti   depressants.

- It's a linear trend with the despositions, from the time we are born to as we get older we have more chance of         needing some form of medication. But the trend narrows down for the one's in their 60s, and continues a flat line       up until the 90's. 

- People in their 50's got the most despostions out of all other age groups. 

- CITALOPRAM was popular in 2010/2011, but with the new research of finding out ESCITALOPRAM was far more efficent,       it's usage began to taper off with a slight increase in usage over 2015/2016, then gradually falling again through     2018.

- VENLAFAXINE had had a steady usage throughout 2010-2018 with a slight increase through 2015 to 2017

- ESCITALOPRAM had increased it's prescriptions signifigantly over the years 2010 to 2018, due to the fact CITALOPRAM
  was found less effective. 

- TRAZODONE had also been on the rise, with more and more prescriptions being filled year after year. Mostly due to the   recent findings that it also works great as a sleeping aid.

- LORAZEPAM was also popular and had a steady growth of prescriptions up until 2017/2018 as usage began to decline.

- VENLAFAXINE and CITALOPRAM were the most prescribed drugs over the years 2010 to 2018 in Edmonton/Calgary.

- VENLAFAXINE is popular among middle age people.

- Despite having a smaller population size than Calgary, Edmonton had a higher number of drug dispositions.


**Venlafaxine** aka Effexor is used to treat depression. It is also used to treat general anxiety disorder, social anxiety disorder, and panic disorder. Venlafaxine belongs to a group of medicines known as serotonin and norepinephrine reuptake inhibitors (SNRI). These medicines are thought to work by increasing the activity of a chemical called serotonin in the brain.  _(FDA approved in 1993_)

**Citalopram**, sold under the brand name Celexa among others, is an antidepressant of the selective serotonin reuptake inhibitor class. It is used to treat major depressive disorder, obsessive compulsive disorder, panic disorder, and social phobia. The antidepressant effects may take one to four weeks to occur  (_FDA approved in 1998_)


**Escitalopram** aka Lexapro is used to treat depression and generalized anxiety disorder (GAD). It is an antidepressant that belongs to a group of medicines known as selective serotonin reuptake inhibitors (SSRIs). These medicines work by increasing the activity of the chemical serotonin in the brain. (_FDA approved in 2002_)

**Lorazepam**  aska Ativan is used to treat anxiety disorders. It is also used for short-term relief of the symptoms of anxiety or anxiety caused by depression. Lorazepam is a benzodiazepine that works in the brain to relieve symptoms of anxiety. Benzodiazepines are central nervous system (CNS) depressants, which are medicines that slow down the nervous system (FDA approved in 1977)

**Trazadone** aka Desrel is a prescription medicine that’s used to treat depression, and, even more commonly, sleep issues related to a variety of mental and physical illnesses. (_FDA approved in 1981_)



## **Notes**
* As of 2012, reviews had concluded (with caveats in some cases) that escitalopram is modestly superior to citalopram in efficacy and tolerability

* Effexor was discontinued because the newer time-released Effexor XR formula can be taken once daily and causes less nausea than the original formula.(FDA approved in 1997)


* The population excludes members of the Armed Forces, RCMP, inmates in Federal
Penitentiaries, and those who have opted out of the Alberta Health Care Insurance Plan.

* Standard scores are calculated for each of the four indicators as follows:
𝑆𝑆(𝑅̂ ) = 𝑅̂𝑟𝑒𝑔𝑖𝑜𝑛𝑎𝑙 − 𝑅̂𝑝𝑟𝑜𝑣𝑖𝑛𝑐𝑖𝑎𝑙
𝑆𝐸(𝑅̂ )𝑟𝑒𝑔𝑖𝑜𝑛𝑎𝑙
Standard scores allow for comparisons between regions and the provincial average. Standard
Scores are used for colour-coding charts maps.

*Dataset was obtained from Alberta Health.